In [ ]:
dbutils.widgets.text("min_T", "")
dbutils.widgets.text("max_T", "")
dbutils.widgets.text("Dimensions", "")
dbutils.widgets.text("tasks", "")
dbutils.widgets.text("Rows", "")

In [ ]:
import numpy as np
from datetime import datetime
from andi_datasets.datasets_challenge import challenge_theory_dataset
from andi_datasets.datasets_theory import datasets_theory 
from sklearn.model_selection import train_test_split
from tensorflow import keras as kr
from keras import models, layers ,Sequential
from keras.layers import Dense,Conv1D,Flatten,BatchNormalization,MaxPooling1D,Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.callbacks import ModelCheckpoint


In [ ]:
min_t=199
max_t=200
task=2
dim=1
row=1000000

print(min_t)
print(max_t)
print(task)
print(dim)
print(row)


199
200
2
1
1000000


In [ ]:
X1_val,Y1_val, X2_val, Y2_val,X3_val,Y3_val=challenge_theory_dataset(min_T = 199, max_T = 200,N = 20000,dimensions = 1,tasks = 2,save_dataset = False)

X1,Y1, X2, Y2,X3,Y3=challenge_theory_dataset(min_T = 199, max_T = 200,dimensions = 1,tasks = 2,load_dataset = True)
AD = datasets_theory()

Creating a dataset for task(s) 2 and dimension(s) 1.
Generating dataset for dimension 1.


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/andi_datasets/models_theory.py:95: RuntimeWarning: overflow encountered in power
  dt = (1-np.random.rand(T))**(-1/sigma)


Creating a dataset for task(s) 2 and dimension(s) 1.


In [ ]:
type(y_test)

list

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2[0],Y2[0], test_size=.2, random_state=1)

X2_val, X_test_val, Y2_val, y_test_val = train_test_split(X2_val[0],Y2_val[0], test_size=.2, random_state=1)

X_train_re = np.array(X_train)
y_train_cat = to_categorical(y_train, num_classes=5)
X_test_re = np.array(X_test)
y_test_cat = to_categorical(y_test, num_classes=5)
X_val_re = np.array(X2_val)
y_val_cat = to_categorical(Y2_val, num_classes=5)

In [ ]:
print("Train X:"+str(len(X_train)))
print("Train Y:"+str(len(y_train)))
print("Test X:"+str(len(X_test)))
print("Test Y:"+str(len(y_test)))
print("Val X:"+str(len(X_val_re)))
print("Val Y:"+str(len(y_val_cat)))

Train X:80000
Train Y:80000
Test X:20000
Test Y:20000
Val X:16000
Val Y:16000


In [ ]:
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Calcular los pesos para cada clase para balancear el entrenamiento
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Creación del modelo secuencial
model = Sequential(name="Modelo_Convolucional")

# Añadir capa convolucional 1D como capa de entrada
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(199, 1), name='Conv1D_entrada'))

# Añadir otra capa convolucional 1D
model.add(Conv1D(filters=64, activation='relu', kernel_size=5, name='Conv1D_2'))
# Añadir una capa de Max Pooling para reducir la dimensionalidad
model.add(MaxPooling1D(pool_size=2, name='MaxPooling1D_1'))
# Añadir Dropout para regularización, evitar el sobreajuste
model.add(Dropout(0.5, name='Dropout_1'))

# Continuar con más capas convolucionales y de pooling
model.add(Conv1D(filters=128, activation='relu', kernel_size=3, name='Conv1D_3'))
model.add(Conv1D(filters=128, activation='relu', kernel_size=3, name='Conv1D_4'))
model.add(MaxPooling1D(pool_size=2, name='MaxPooling1D_2'))
model.add(Dropout(0.5, name='Dropout_2'))

# Aplanar los datos para poder conectarlos con capas densas
model.add(Flatten(name='Flatten'))

# Añadir capas densas para clasificación
model.add(Dense(128, activation='relu', name='Dense_1'))
model.add(Dense(5, activation='softmax', name='Dense_salida'))  # 5 clases

# Configurar el optimizador, en este caso Adam con tasa de aprendizaje y norma máxima
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
# Compilar el modelo con función de pérdida y métricas
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Añadir callbacks para el entrenamiento: parada temprana y guardar el mejor modelo
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Entrenar el modelo con los datos de entrenamiento y validación, usando los pesos de clase y callbacks definidos

history = model.fit(X_train_re, y_train_cat, epochs=100, batch_size=32, validation_data=(X_val_re, y_val_cat), class_weight=class_weight_dict, callbacks=[es, mc])

In [ ]:
from keras.models import load_model

# Cargar el modelo entrenado
loaded_model = load_model('best_model.h5')




In [ ]:

y_pred = loaded_model.predict(X_test_re) 

print('accuracy: ',accuracy_score(np.array(y_test),np.argmax(y_pred, axis = 1)))
print('f1_score ',f1_score(np.array(y_test),np.argmax(y_pred, axis = 1), average='micro'))
print("Accuracy:", accuracy_score(np.array(y_test), np.argmax(y_pred, axis = 1)))
print("\nClassification Report:\n", classification_report(np.array(y_test), np.argmax(y_pred, axis = 1)))
print("\nConfusion Matrix:\n", confusion_matrix(np.array(y_test), np.argmax(y_pred, axis = 1)))



In [ ]:
# Import required libraries
from sklearn.metrics import confusion_matrix

# Generate the confusion matrix
cm = confusion_matrix(np.array(y_test), np.argmax(y_pred, axis = 1))

# Print the confusion matrix
print(cm)


[[1413  668  552   27 1281]
 [ 181 3691  142    0   12]
 [  45  296 2502   96 1099]
 [   1    4   16 3925   33]
 [ 161    6  840   58 2951]]


In [ ]:
print(confusion_matrix(np.array(y_test),np.argmax(y_pred, axis = 1)))

[[18 37 29  1 20]
 [13 69 31  4 15]
 [10 40 68  1 21]
 [ 6 12  9 82 16]
 [20 26 29  6 30]]


In [ ]:
df = spark.read.format("mlflow-experiment").load()

display(df)

In [ ]:

confusion_matrix=cm
# Calcular las métricas de rendimiento
# Verdaderos Positivos (diagonal de la matriz)
TP = np.diag(confusion_matrix)

# Falsos Positivos (suma de las columnas de la matriz - Verdaderos Positivos)
FP = np.sum(confusion_matrix, axis=0) - TP

# Falsos Negativos (suma de las filas de la matriz - Verdaderos Positivos)
FN = np.sum(confusion_matrix, axis=1) - TP

# Verdaderos Negativos (suma total de la matriz - (FP + FN + TP))
TN = np.sum(confusion_matrix) - (FP + FN + TP)

# Precisión (Precision)
precision = TP / (TP + FP)

# Sensibilidad (Recall)
recall = TP / (TP + FN)

# Exactitud (Accuracy)
accuracy = np.sum(TP) / np.sum(confusion_matrix)

performance_metrics = {
    "TP": TP,
    "FP": FP,
    "FN": FN,
    "TN": TN,
    "Precision": precision,
    "Recall": recall,
    "Accuracy": accuracy
}

performance_metrics


{'TP': array([1413, 3691, 2502, 3925, 2951]),
 'FP': array([ 388,  974, 1550,  181, 2425]),
 'FN': array([2528,  335, 1536,   54, 1065]),
 'TN': array([15671, 15000, 14412, 15840, 13559]),
 'Precision': array([0.78456413, 0.79121115, 0.61747285, 0.95591817, 0.54892113]),
 'Recall': array([0.35853844, 0.91679086, 0.61961367, 0.98642875, 0.73481076]),
 'Accuracy': 0.8262}